<a href="https://colab.research.google.com/github/BhavyaaaJain/HSCode/blob/main/Final2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow_text
!pip install sentence-transformers
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
import spacy
from sentence_transformers import SentenceTransformer, util

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=c82bd01233998d48b7820f8dc69b0143a34ccf3216516b90bfae154a315a781e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-tr

In [2]:
df = pd.read_excel("data.xlsx")

In [3]:
df

,Chapter Number,TariffItem,Description of goods,IGST,Unit,Unnamed: 5
0,CHAPTER 1,1012100,"Live Horses, Asses, Mules and Hinnies Horses:...",0.12,u,NaN
1,CHAPTER 1,10129,"Live Horses, Asses, Mules and Hinnies Horses:...",NaN,NaN,NaN
2,CHAPTER 1,1012910,"Live Horses, Asses, Mules and Hinnies Horses:...",0.12,u,NaN
3,CHAPTER 1,1012990,"Live Horses, Asses, Mules and Hinnies Horses:...",0.12,u,NaN
4,CHAPTER 1,10130,"Live Horses, Asses, Mules and Hinnies Asses:",NaN,NaN,NaN
...,...,...,...,...,...,...
13062,CHAPTER 97,97052900,Collections and collectors' pieces of archaeol...,0.12,u,NaN
13063,CHAPTER 97,97053100,Collections and collectors' pieces of archaeol...,0.12,u,NaN
13064,CHAPTER 97,97053900,Collections and collectors' pieces of archaeol...,0.12,u,NaN
13065,CHAPTER 97,97061000,Antiques of an age exceeding 100 years Of an ...,0.12,u,NaN


In [4]:
df=df.drop(['IGST', 'Unit', 'Unnamed: 5'], axis=1)
df

,Chapter Number,TariffItem,Description of goods
0,CHAPTER 1,1012100,"Live Horses, Asses, Mules and Hinnies Horses:..."
1,CHAPTER 1,10129,"Live Horses, Asses, Mules and Hinnies Horses:..."
2,CHAPTER 1,1012910,"Live Horses, Asses, Mules and Hinnies Horses:..."
3,CHAPTER 1,1012990,"Live Horses, Asses, Mules and Hinnies Horses:..."
4,CHAPTER 1,10130,"Live Horses, Asses, Mules and Hinnies Asses:"
...,...,...,...
13062,CHAPTER 97,97052900,Collections and collectors' pieces of archaeol...
13063,CHAPTER 97,97053100,Collections and collectors' pieces of archaeol...
13064,CHAPTER 97,97053900,Collections and collectors' pieces of archaeol...
13065,CHAPTER 97,97061000,Antiques of an age exceeding 100 years Of an ...


In [5]:
df.columns=['chapter', 'code', 'desc']
df

,chapter,code,desc
0,CHAPTER 1,1012100,"Live Horses, Asses, Mules and Hinnies Horses:..."
1,CHAPTER 1,10129,"Live Horses, Asses, Mules and Hinnies Horses:..."
2,CHAPTER 1,1012910,"Live Horses, Asses, Mules and Hinnies Horses:..."
3,CHAPTER 1,1012990,"Live Horses, Asses, Mules and Hinnies Horses:..."
4,CHAPTER 1,10130,"Live Horses, Asses, Mules and Hinnies Asses:"
...,...,...,...
13062,CHAPTER 97,97052900,Collections and collectors' pieces of archaeol...
13063,CHAPTER 97,97053100,Collections and collectors' pieces of archaeol...
13064,CHAPTER 97,97053900,Collections and collectors' pieces of archaeol...
13065,CHAPTER 97,97061000,Antiques of an age exceeding 100 years Of an ...


In [6]:
#give ids to each chapter
chapter_mapping = {i: chapter for i, chapter in enumerate(df['chapter'].unique())}

In [7]:
df.head()

,chapter,code,desc
0,CHAPTER 1,1012100,"Live Horses, Asses, Mules and Hinnies Horses:..."
1,CHAPTER 1,10129,"Live Horses, Asses, Mules and Hinnies Horses:..."
2,CHAPTER 1,1012910,"Live Horses, Asses, Mules and Hinnies Horses:..."
3,CHAPTER 1,1012990,"Live Horses, Asses, Mules and Hinnies Horses:..."
4,CHAPTER 1,10130,"Live Horses, Asses, Mules and Hinnies Asses:"


In [8]:
model = SentenceTransformer('paraphrase-albert-small-v2')

In [9]:
import torch
emb = []
# Split the first sentence into words
common_part = df["desc"].loc[0].split()
common_part_final = common_part
fi = 0
en = 0
# Initialize the common part as the first sentence
cnt=0
# Loop through the rest of the sentences to find the common part
for index, sentence in enumerate(df["desc"].iloc[1:]):
    words = sentence.split()
    i = 0
    while i < len(common_part) and i < len(words) and common_part[i] == words[i]:
        i += 1
    common_part = common_part[:i]
    if len(common_part)>=3:
      common_part_final = common_part
      en = index
    else:
      common_part_str = " ".join(common_part_final)
      embeddings = model.encode(common_part_str, convert_to_tensor=True)

      if cnt==0:
        df.loc[0, "common_part"] = common_part_str
        # embeddings = pd.Series([embeddings]*(1))
        emb.append(embeddings)
        # df.loc[0, "common_embeddings"] = embeddings.values
        cnt+=1

      df.loc[fi + 1:en + 1, "common_part"] = common_part_str
      for i in range(fi, en + 1):  # Add 1 to end_index to make it inclusive
        emb.append(embeddings)

      # embeddings = pd.Series([embeddings]*(en-fi+1))
      # df.loc[fi + 1:en + 1, "common_embeddings"] = embeddings.values
      fi = index
      en = index
      # print(df)
      common_part = sentence.split()
      common_part_final = common_part

      print(index)

common_part_str = " ".join(common_part_final)
embeddings = model.encode(common_part_str, convert_to_tensor=True)
print(embeddings)
# print(embeddings.dtype())
for i in range(fi, en + 1):  # Add 1 to end_index to make it inclusive
        emb.append(embeddings)
# embeddings = pd.Series([embeddings]*(en-fi+1))
df.loc[fi + 1:en + 1, "common_part"] = common_part_str
# df.loc[fi + 1:en + 1, "common_embeddings"] =embeddings.values
df['common_embeddings'] = emb
df

10
20
21
23
27
34
51
57
58
59
60
61
62
63
64
70
73
74
78
79
80
81
101
108
114
120
122
134
136
143
144
171
195
200
211
214
221
231
235
238
242
247
253
257
263
276
286
303
322
323
324
326
337
344
346
347
355
359
369
373
391
409
422
437
441
452
456
462
464
474
481
492
500
514
520
526
535
548
567
575
577
585
587
589
593
605
613
643
662
677
683
692
696
710
716
726
740
742
784
788
795
801
804
855
863
884
892
893
898
916
922
926
929
933
936
942
949
954
976
986
991
996
1002
1010
1017
1025
1027
1034
1041
1044
1047
1060
1061
1062
1072
1079
1097
1102
1107
1113
1128
1147
1156
1160
1166
1168
1170
1176
1180
1186
1189
1206
1208
1217
1219
1222
1228
1231
1238
1242
1300
1310
1320
1328
1334
1337
1342
1346
1350
1370
1375
1378
1379
1380
1383
1389
1407
1420
1425
1427
1433
1437
1440
1446
1448
1449
1450
1454
1458
1461
1465
1488
1491
1497
1503
1504
1505
1506
1507
1521
1533
1543
1547
1553
1557
1561
1565
1579
1580
1581
1582
1583
1589
1592
1598
1617
1632
1644
1649
1655
1670
1675
1687
1702
1709
1717
1725
1732
1734

,chapter,code,desc,common_part,common_embeddings
0,CHAPTER 1,1012100,"Live Horses, Asses, Mules and Hinnies Horses:...","Live Horses, Asses, Mules and Hinnies","[tensor(-0.2544), tensor(-0.3045), tensor(0.75..."
1,CHAPTER 1,10129,"Live Horses, Asses, Mules and Hinnies Horses:...","Live Horses, Asses, Mules and Hinnies","[tensor(-0.2544), tensor(-0.3045), tensor(0.75..."
2,CHAPTER 1,1012910,"Live Horses, Asses, Mules and Hinnies Horses:...","Live Horses, Asses, Mules and Hinnies","[tensor(-0.2544), tensor(-0.3045), tensor(0.75..."
3,CHAPTER 1,1012990,"Live Horses, Asses, Mules and Hinnies Horses:...","Live Horses, Asses, Mules and Hinnies","[tensor(-0.2544), tensor(-0.3045), tensor(0.75..."
4,CHAPTER 1,10130,"Live Horses, Asses, Mules and Hinnies Asses:","Live Horses, Asses, Mules and Hinnies","[tensor(-0.2544), tensor(-0.3045), tensor(0.75..."
...,...,...,...,...,...
13062,CHAPTER 97,97052900,Collections and collectors' pieces of archaeol...,Collections and collectors' pieces of archaeol...,"[tensor(-0.2374), tensor(0.0589), tensor(-0.23..."
13063,CHAPTER 97,97053100,Collections and collectors' pieces of archaeol...,Collections and collectors' pieces of archaeol...,"[tensor(-0.2374), tensor(0.0589), tensor(-0.23..."
13064,CHAPTER 97,97053900,Collections and collectors' pieces of archaeol...,Collections and collectors' pieces of archaeol...,"[tensor(-0.2374), tensor(0.0589), tensor(-0.23..."
13065,CHAPTER 97,97061000,Antiques of an age exceeding 100 years Of an ...,Antiques of an age exceeding 100 years,"[tensor(-0.5290), tensor(-0.2318), tensor(0.15..."


In [10]:
df['common_embeddings'] = emb
df

,chapter,code,desc,common_part,common_embeddings
0,CHAPTER 1,1012100,"Live Horses, Asses, Mules and Hinnies Horses:...","Live Horses, Asses, Mules and Hinnies","[tensor(-0.2544), tensor(-0.3045), tensor(0.75..."
1,CHAPTER 1,10129,"Live Horses, Asses, Mules and Hinnies Horses:...","Live Horses, Asses, Mules and Hinnies","[tensor(-0.2544), tensor(-0.3045), tensor(0.75..."
2,CHAPTER 1,1012910,"Live Horses, Asses, Mules and Hinnies Horses:...","Live Horses, Asses, Mules and Hinnies","[tensor(-0.2544), tensor(-0.3045), tensor(0.75..."
3,CHAPTER 1,1012990,"Live Horses, Asses, Mules and Hinnies Horses:...","Live Horses, Asses, Mules and Hinnies","[tensor(-0.2544), tensor(-0.3045), tensor(0.75..."
4,CHAPTER 1,10130,"Live Horses, Asses, Mules and Hinnies Asses:","Live Horses, Asses, Mules and Hinnies","[tensor(-0.2544), tensor(-0.3045), tensor(0.75..."
...,...,...,...,...,...
13062,CHAPTER 97,97052900,Collections and collectors' pieces of archaeol...,Collections and collectors' pieces of archaeol...,"[tensor(-0.2374), tensor(0.0589), tensor(-0.23..."
13063,CHAPTER 97,97053100,Collections and collectors' pieces of archaeol...,Collections and collectors' pieces of archaeol...,"[tensor(-0.2374), tensor(0.0589), tensor(-0.23..."
13064,CHAPTER 97,97053900,Collections and collectors' pieces of archaeol...,Collections and collectors' pieces of archaeol...,"[tensor(-0.2374), tensor(0.0589), tensor(-0.23..."
13065,CHAPTER 97,97061000,Antiques of an age exceeding 100 years Of an ...,Antiques of an age exceeding 100 years,"[tensor(-0.5290), tensor(-0.2318), tensor(0.15..."


In [11]:
for index, row in df.iterrows():
  len_common = len(row['common_part'])
  rem = row['desc'][len_common: ]
  df.at[index, 'remaining'] = rem

df

,chapter,code,desc,common_part,common_embeddings,remaining
0,CHAPTER 1,1012100,"Live Horses, Asses, Mules and Hinnies Horses:...","Live Horses, Asses, Mules and Hinnies","[tensor(-0.2544), tensor(-0.3045), tensor(0.75...",Horses: Pure-bred breeding animals
1,CHAPTER 1,10129,"Live Horses, Asses, Mules and Hinnies Horses:...","Live Horses, Asses, Mules and Hinnies","[tensor(-0.2544), tensor(-0.3045), tensor(0.75...",Horses: Other:
2,CHAPTER 1,1012910,"Live Horses, Asses, Mules and Hinnies Horses:...","Live Horses, Asses, Mules and Hinnies","[tensor(-0.2544), tensor(-0.3045), tensor(0.75...",Horses: Other: Horses for polo
3,CHAPTER 1,1012990,"Live Horses, Asses, Mules and Hinnies Horses:...","Live Horses, Asses, Mules and Hinnies","[tensor(-0.2544), tensor(-0.3045), tensor(0.75...",Horses: Other: Other
4,CHAPTER 1,10130,"Live Horses, Asses, Mules and Hinnies Asses:","Live Horses, Asses, Mules and Hinnies","[tensor(-0.2544), tensor(-0.3045), tensor(0.75...",Asses:
...,...,...,...,...,...,...
13062,CHAPTER 97,97052900,Collections and collectors' pieces of archaeol...,Collections and collectors' pieces of archaeol...,"[tensor(-0.2374), tensor(0.0589), tensor(-0.23...","f zoological, botanical, mineralogical, anatom..."
13063,CHAPTER 97,97053100,Collections and collectors' pieces of archaeol...,Collections and collectors' pieces of archaeol...,"[tensor(-0.2374), tensor(0.0589), tensor(-0.23...",f numismatic interest: Of an age exceeding 100...
13064,CHAPTER 97,97053900,Collections and collectors' pieces of archaeol...,Collections and collectors' pieces of archaeol...,"[tensor(-0.2374), tensor(0.0589), tensor(-0.23...",f numismatic interest: Other
13065,CHAPTER 97,97061000,Antiques of an age exceeding 100 years Of an ...,Antiques of an age exceeding 100 years,"[tensor(-0.5290), tensor(-0.2318), tensor(0.15...",Of an age exceeding 250 years


In [12]:
df.to_csv("encoded_data.csv")

In [13]:
remaining_embeddings = []
for index, row in df.iterrows():
    embedding = model.encode(row['remaining'], convert_to_tensor=True)
    remaining_embeddings.append(embedding)
    print(index, embedding)
df['remaining_embeddings'] = remaining_embeddings
df

Streaming output truncated to the last 5000 lines.
        -4.9385e-01,  2.0689e-01,  5.9811e-01, -6.7084e-01,  3.3394e-01,
        -1.1248e+00,  2.9482e-02,  2.2163e-01,  1.0062e-01,  3.9076e-01,
         1.3100e-01, -3.1412e-01,  4.3404e-01, -2.9268e-01, -8.6572e-01,
        -1.0282e+00,  2.2128e-01,  1.1536e-01, -2.3354e-01,  1.3117e-01,
         1.3614e-01, -1.7072e-01,  6.0107e-01,  5.2151e-01, -2.6390e-01,
         1.9803e-01,  5.4973e-01,  1.1290e-01,  5.5021e-01,  1.2502e-02,
         1.9439e-01, -4.5450e-01, -2.9925e-01,  6.6197e-01,  2.5470e-01,
        -2.9101e-01, -1.8026e-01,  4.1446e-01, -3.0079e-01, -2.2735e-01,
         6.4917e-01, -7.0902e-01,  1.3235e-01,  6.0894e-01, -2.7834e-02,
        -9.3582e-02, -2.9446e-01,  2.6512e-01,  9.5628e-02,  1.7140e-01,
        -1.5771e+00,  8.3328e-01,  5.2210e-01, -4.6866e-01, -1.5124e-01,
         2.5767e-01, -1.3499e-01, -8.9319e-02, -5.5424e-01, -1.5695e-01,
         6.8305e-02, -5.0961e-02, -2.8170e-01,  5.4438e-01, -1.9064e-01,


,chapter,code,desc,common_part,common_embeddings,remaining,remaining_embeddings
0,CHAPTER 1,1012100,"Live Horses, Asses, Mules and Hinnies Horses:...","Live Horses, Asses, Mules and Hinnies","[tensor(-0.2544), tensor(-0.3045), tensor(0.75...",Horses: Pure-bred breeding animals,"[tensor(0.7281), tensor(0.3046), tensor(0.3030..."
1,CHAPTER 1,10129,"Live Horses, Asses, Mules and Hinnies Horses:...","Live Horses, Asses, Mules and Hinnies","[tensor(-0.2544), tensor(-0.3045), tensor(0.75...",Horses: Other:,"[tensor(-0.0426), tensor(-0.0908), tensor(0.90..."
2,CHAPTER 1,1012910,"Live Horses, Asses, Mules and Hinnies Horses:...","Live Horses, Asses, Mules and Hinnies","[tensor(-0.2544), tensor(-0.3045), tensor(0.75...",Horses: Other: Horses for polo,"[tensor(-0.0349), tensor(0.2209), tensor(0.407..."
3,CHAPTER 1,1012990,"Live Horses, Asses, Mules and Hinnies Horses:...","Live Horses, Asses, Mules and Hinnies","[tensor(-0.2544), tensor(-0.3045), tensor(0.75...",Horses: Other: Other,"[tensor(0.0435), tensor(-0.1021), tensor(0.804..."
4,CHAPTER 1,10130,"Live Horses, Asses, Mules and Hinnies Asses:","Live Horses, Asses, Mules and Hinnies","[tensor(-0.2544), tensor(-0.3045), tensor(0.75...",Asses:,"[tensor(-0.0660), tensor(0.7764), tensor(-0.87..."
...,...,...,...,...,...,...,...
13062,CHAPTER 97,97052900,Collections and collectors' pieces of archaeol...,Collections and collectors' pieces of archaeol...,"[tensor(-0.2374), tensor(0.0589), tensor(-0.23...","f zoological, botanical, mineralogical, anatom...","[tensor(-0.3126), tensor(0.3570), tensor(0.246..."
13063,CHAPTER 97,97053100,Collections and collectors' pieces of archaeol...,Collections and collectors' pieces of archaeol...,"[tensor(-0.2374), tensor(0.0589), tensor(-0.23...",f numismatic interest: Of an age exceeding 100...,"[tensor(0.2381), tensor(0.2114), tensor(0.2482..."
13064,CHAPTER 97,97053900,Collections and collectors' pieces of archaeol...,Collections and collectors' pieces of archaeol...,"[tensor(-0.2374), tensor(0.0589), tensor(-0.23...",f numismatic interest: Other,"[tensor(0.2676), tensor(0.3764), tensor(0.5317..."
13065,CHAPTER 97,97061000,Antiques of an age exceeding 100 years Of an ...,Antiques of an age exceeding 100 years,"[tensor(-0.5290), tensor(-0.2318), tensor(0.15...",Of an age exceeding 250 years,"[tensor(0.2543), tensor(-0.0003), tensor(0.143..."


In [14]:
df=df.drop(['common_part', 'remaining'], axis=1)
df

,chapter,code,desc,common_embeddings,remaining_embeddings
0,CHAPTER 1,1012100,"Live Horses, Asses, Mules and Hinnies Horses:...","[tensor(-0.2544), tensor(-0.3045), tensor(0.75...","[tensor(0.7281), tensor(0.3046), tensor(0.3030..."
1,CHAPTER 1,10129,"Live Horses, Asses, Mules and Hinnies Horses:...","[tensor(-0.2544), tensor(-0.3045), tensor(0.75...","[tensor(-0.0426), tensor(-0.0908), tensor(0.90..."
2,CHAPTER 1,1012910,"Live Horses, Asses, Mules and Hinnies Horses:...","[tensor(-0.2544), tensor(-0.3045), tensor(0.75...","[tensor(-0.0349), tensor(0.2209), tensor(0.407..."
3,CHAPTER 1,1012990,"Live Horses, Asses, Mules and Hinnies Horses:...","[tensor(-0.2544), tensor(-0.3045), tensor(0.75...","[tensor(0.0435), tensor(-0.1021), tensor(0.804..."
4,CHAPTER 1,10130,"Live Horses, Asses, Mules and Hinnies Asses:","[tensor(-0.2544), tensor(-0.3045), tensor(0.75...","[tensor(-0.0660), tensor(0.7764), tensor(-0.87..."
...,...,...,...,...,...
13062,CHAPTER 97,97052900,Collections and collectors' pieces of archaeol...,"[tensor(-0.2374), tensor(0.0589), tensor(-0.23...","[tensor(-0.3126), tensor(0.3570), tensor(0.246..."
13063,CHAPTER 97,97053100,Collections and collectors' pieces of archaeol...,"[tensor(-0.2374), tensor(0.0589), tensor(-0.23...","[tensor(0.2381), tensor(0.2114), tensor(0.2482..."
13064,CHAPTER 97,97053900,Collections and collectors' pieces of archaeol...,"[tensor(-0.2374), tensor(0.0589), tensor(-0.23...","[tensor(0.2676), tensor(0.3764), tensor(0.5317..."
13065,CHAPTER 97,97061000,Antiques of an age exceeding 100 years Of an ...,"[tensor(-0.5290), tensor(-0.2318), tensor(0.15...","[tensor(0.2543), tensor(-0.0003), tensor(0.143..."


In [15]:
df.to_csv("final_data.csv")

In [28]:
br = pd.read_csv("brand.csv")
br.drop(['Unnamed: 0'], axis=1)
br

,Unnamed: 0,Brand,desc
0,0,Fresho,"The category is Fruits & Vegetables, subcatego..."
1,1,Fresho,"The category is Fruits & Vegetables, subcatego..."
2,2,Fresho,"The category is Fruits & Vegetables, subcatego..."
3,3,Fresho,"The category is Fruits & Vegetables, subcatego..."
4,4,Fresho,"The category is Fruits & Vegetables, subcatego..."
...,...,...,...
9668,9668,Noir,The category is Home&Kitchen|Kitchen&HomeAppli...
9669,9669,Prestige,The category is Home&Kitchen|Kitchen&HomeAppli...
9670,9670,Bajaj,"The category is Home&Kitchen|Heating,Cooling&A..."
9671,9671,Havells,"The category is Home&Kitchen|Heating,Cooling&A..."


In [30]:
name_embeddings = []
desc_embeddings = []
for index, row in br.iterrows():
    name_embedding = model.encode(row['Brand'], convert_to_tensor=True)
    desc_embedding = model.encode(row['desc'], convert_to_tensor=True)
    name_embeddings.append(name_embedding)
    desc_embeddings.append(desc_embedding)
    print(index)
br['brand_embeddings'] = name_embeddings
br['desc_embeddings'] = desc_embeddings

br

Streaming output truncated to the last 5000 lines.
4673
4674
4675
4676
4677
4678
4679
4680
4681
4682
4683
4684
4685
4686
4687
4688
4689
4690
4691
4692
4693
4694
4695
4696
4697
4698
4699
4700
4701
4702
4703
4704
4705
4706
4707
4708
4709
4710
4711
4712
4713
4714
4715
4716
4717
4718
4719
4720
4721
4722
4723
4724
4725
4726
4727
4728
4729
4730
4731
4732
4733
4734
4735
4736
4737
4738
4739
4740
4741
4742
4743
4744
4745
4746
4747
4748
4749
4750
4751
4752
4753
4754
4755
4756
4757
4758
4759
4760
4761
4762
4763
4764
4765
4766
4767
4768
4769
4770
4771
4772
4773
4774
4775
4776
4777
4778
4779
4780
4781
4782
4783
4784
4785
4786
4787
4788
4789
4790
4791
4792
4793
4794
4795
4796
4797
4798
4799
4800
4801
4802
4803
4804
4805
4806
4807
4808
4809
4810
4811
4812
4813
4814
4815
4816
4817
4818
4819
4820
4821
4822
4823
4824
4825
4826
4827
4828
4829
4830
4831
4832
4833
4834
4835
4836
4837
4838
4839
4840
4841
4842
4843
4844
4845
4846
4847
4848
4849
4850
4851
4852
4853
4854
4855
4856
4857
4858
4859
4860
4861
4862

,Unnamed: 0,Brand,desc,brand_embeddings,desc_embeddings
0,0,Fresho,"The category is Fruits & Vegetables, subcatego...","[tensor(0.9063), tensor(0.2331), tensor(-0.114...","[tensor(-0.0749), tensor(0.4582), tensor(-0.02..."
1,1,Fresho,"The category is Fruits & Vegetables, subcatego...","[tensor(0.9063), tensor(0.2331), tensor(-0.114...","[tensor(-0.1485), tensor(0.5059), tensor(-0.04..."
2,2,Fresho,"The category is Fruits & Vegetables, subcatego...","[tensor(0.9063), tensor(0.2331), tensor(-0.114...","[tensor(-0.0749), tensor(0.4582), tensor(-0.02..."
3,3,Fresho,"The category is Fruits & Vegetables, subcatego...","[tensor(0.9063), tensor(0.2331), tensor(-0.114...","[tensor(-0.1485), tensor(0.5059), tensor(-0.04..."
4,4,Fresho,"The category is Fruits & Vegetables, subcatego...","[tensor(0.9063), tensor(0.2331), tensor(-0.114...","[tensor(-0.0749), tensor(0.4582), tensor(-0.02..."
...,...,...,...,...,...
9668,9668,Noir,The category is Home&Kitchen|Kitchen&HomeAppli...,"[tensor(-0.0292), tensor(-0.5769), tensor(-0.4...","[tensor(0.4568), tensor(0.1761), tensor(0.1447..."
9669,9669,Prestige,The category is Home&Kitchen|Kitchen&HomeAppli...,"[tensor(-0.2247), tensor(-0.4807), tensor(0.41...","[tensor(-0.1181), tensor(0.0424), tensor(-0.05..."
9670,9670,Bajaj,"The category is Home&Kitchen|Heating,Cooling&A...","[tensor(-0.1541), tensor(0.1540), tensor(1.030...","[tensor(0.0741), tensor(-0.0291), tensor(0.054..."
9671,9671,Havells,"The category is Home&Kitchen|Heating,Cooling&A...","[tensor(0.4351), tensor(0.4712), tensor(0.3816...","[tensor(0.3964), tensor(0.0647), tensor(-0.097..."


In [32]:
br=br.drop(['Brand', 'desc'], axis=1)
br

,Unnamed: 0,brand_embeddings,desc_embeddings
0,0,"[tensor(0.9063), tensor(0.2331), tensor(-0.114...","[tensor(-0.0749), tensor(0.4582), tensor(-0.02..."
1,1,"[tensor(0.9063), tensor(0.2331), tensor(-0.114...","[tensor(-0.1485), tensor(0.5059), tensor(-0.04..."
2,2,"[tensor(0.9063), tensor(0.2331), tensor(-0.114...","[tensor(-0.0749), tensor(0.4582), tensor(-0.02..."
3,3,"[tensor(0.9063), tensor(0.2331), tensor(-0.114...","[tensor(-0.1485), tensor(0.5059), tensor(-0.04..."
4,4,"[tensor(0.9063), tensor(0.2331), tensor(-0.114...","[tensor(-0.0749), tensor(0.4582), tensor(-0.02..."
...,...,...,...
9668,9668,"[tensor(-0.0292), tensor(-0.5769), tensor(-0.4...","[tensor(0.4568), tensor(0.1761), tensor(0.1447..."
9669,9669,"[tensor(-0.2247), tensor(-0.4807), tensor(0.41...","[tensor(-0.1181), tensor(0.0424), tensor(-0.05..."
9670,9670,"[tensor(-0.1541), tensor(0.1540), tensor(1.030...","[tensor(0.0741), tensor(-0.0291), tensor(0.054..."
9671,9671,"[tensor(0.4351), tensor(0.4712), tensor(0.3816...","[tensor(0.3964), tensor(0.0647), tensor(-0.097..."


In [33]:
br.to_csv("final_brand.csv")

In [46]:
def brand_sim(input_embedding, threshold, df):
  indexes=[]
  for index, row, in br.iterrows():
    cosine_scores = util.cos_sim(row['brand_embeddings'], input_embedding)
    if cosine_scores>=threshold:
      indexes.append(index)

  # print(indexes)
  return indexes


In [48]:
def calculate_sim(input_embedding, threshold):
  chapters=[]
  for index, row in df.iterrows():
    # print(row['common_embeddings'].numpy().dtype())
    cosine_scores = util.cos_sim(row['common_embeddings'], input_embedding)
    # print(cosine_scores)
    if cosine_scores>=threshold:
      if row['chapter'] not in chapters:
        chapters.append(row['chapter'])
  print(chapters)
  return chapters

In [63]:
def top_brand_sim(indexes, br, df):
  top_similarities = []
  for index in indexes:
    for ind, row in df.iterrows():
      sim = util.cos_sim(br.loc[index, 'desc_embeddings'], row['common_embeddings'])
      # print(sim)
      if sim>=0.4:
        sim = util.cos_sim(br.loc[index, 'desc_embeddings'], row['remaining_embeddings'])
        if sim>=0.3:
          top_similarities.append((row['code'], row['desc'], row['chapter'], sim))

  top_similarities.sort(key=lambda x: x[3], reverse=True)
  # print(top_similarities[: 5])
  return top_similarities[: 2]

In [42]:
def calc_sim_rem(input_embedding, chapters):

  top_similarities = []
  for ch in chapters:
    chapter_data = df[df['chapter']==ch]
    similarities = []
    for index, row in chapter_data.iterrows():
      sim = util.cos_sim(row['remaining_embeddings'], input_embedding)
      similarities.append((row['code'], row['desc'], row['chapter'], sim))
    similarities.sort(key=lambda x: x[3], reverse=True)

    if len(chapters)>=5:
      top_similarities.extend(similarities[:1])
    else:
      top_similarities.extend(similarities[:2])
    # print(top_similarities)

  top_similarities.sort(key=lambda x: x[3], reverse=True)
  return top_similarities



In [62]:
input = "surf excel"
input_embedding = model.encode(input, convert_to_tensor=True)
print(input)
thresh=0.5
while len(calculate_sim(input_embedding, thresh))==0 and thresh>=0.15:
  thresh-=0.05

chapters = calculate_sim(input_embedding, thresh)
indexes = brand_sim(input_embedding, 0.7, br)

top_brand_sim = top_brand_sim(indexes, br, df)


top_sim = calc_sim_rem(input_embedding, chapters)
for data in top_brand_sim:
  print(data)
for data in top_sim:
  print(data)



surf excel
[]
[]
[]
[]
['CHAPTER 48', 'CHAPTER 84', 'CHAPTER 96']
['CHAPTER 48', 'CHAPTER 84', 'CHAPTER 96']
[648, 649, 650, 651]
('84565000', 'Machine-tools for working any material by removal of material, by laser or other light or photon beam, ultra-sonic, electro-discharge, electro chemical, electron beam, ionic-beam or plasma arc processes; water-jet cutting machines  Water-jet cutting machines', 'CHAPTER 84', tensor([[0.4024]]))
('48062000', 'Vegetable parchment, greaseproof papers, tracing papers and glassine and other glazed transparent or translucent papers, in rolls or sheets  Greaseproof papers ', 'CHAPTER 48', tensor([[0.3886]]))
('84639020', 'Other machine-tools for working metal, or cermets, without removing material  Other: Strip profiling ', 'CHAPTER 84', tensor([[0.3770]]))
('48054000', 'Other uncoated paper and paperboard, in rolls or sheets, not further worked or processed than as specified in Note 3 to this Chapter  Filter paper and paperboard', 'CHAPTER 48', tensor

In [ ]:
# def calc_sim_rem(input_embedding, chapters):

#   top_similarities = []
#   for ch in chapters:
#     chapter_data = df[df['chapter']==ch]
#     similarities = []
#     for index, row in chapter_data.iterrows():
#       sim = util.cos_sim(row['remaining_embeddings'], input_embedding)
#       similarities.append((row['code'], row['desc'], row['chapter'], sim))
#     similarities.sort(key=lambda x: x[3], reverse=True)

#     if len(chapters)>=5:
#       top_similarities.extend(similarities[:1])
#     else:
#       top_similarities.extend(similarities[:2])
#     # print(top_similarities)

#   top_similarities.sort(key=lambda x: x[3], reverse=True)
#   return top_similarities



In [ ]:
# top_sim = calc_sim_rem(input_embedding, chapters)
# for data in top_sim:
#   print(data)
# print(len(top_sim))